In [1]:
%matplotlib qt
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import colors
from matplotlib import cm
from matplotlib.ticker import NullFormatter
from mpl_toolkits.mplot3d import Axes3D

from scipy import stats
from scipy.stats import multivariate_normal

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from skimage.transform import resize

import math

import seaborn as sns; sns.set()
np.random.seed(42)

In [2]:
class genere_distributions():
    x_min, x_max = 0., 10.
    y_min, y_max = 0., 10.
    nx, ny = 300, 300
    offset = -0.1
    
    def __init__(self, mu, sigma, angle, prob_C):
        self.mu = mu
        self.sigma = sigma
        self.angle = angle
        self.prob_C = prob_C
        
        self.genere_grid()
        self.genere_pdf()


    def genere_grid(self):
        self.xx, self.yy = np.meshgrid(np.linspace(self.x_min, self.x_max, self.nx), 
                                       np.linspace(self.y_min, self.y_max, self.ny))
        self.pos = np.dstack((self.xx, self.yy))
        


    # Modèles génératifs pour les distributions normales 2D 
    # (PDF: probability distribution functions)
    def genere_pdf(self):
        modeles = [None]*2
        for i in range(2):
            # Matrice de rotation
            theta = np.radians(self.angle[i])
            c, s = np.cos(theta), np.sin(theta)
            R = np.array(((c, -s), (s, c)))        

            # Matrice de covariance sans rotation
            C = np.array([[self.sigma[i, 0]**2, 0.],[0., self.sigma[i, 1]**2]])

            # Matrice de covariance après rotation
            # new_cov = rotation_matrix @ cov @ rotation_matrix.T
            C = R.dot( C.dot(R.T) )  

            # Génère modele pour PDF normale 2D
            modeles[i] = multivariate_normal(self.mu[i,:], C) 

        # Génère PDF individuelles
        pdf0 = self.prob_C[0]*modeles[0].pdf(self.pos)
        pdf1 = self.prob_C[1]*modeles[1].pdf(self.pos)

        # Masque indiquant la zone d'influence du modèle 1
        self.masque = pdf1 > pdf0

        # PDF globale
        self.pdf = pdf0 + pdf1

    
    def genere_surf_3D(self, ax):
        # Génération de la surface 3D en 2 couleurs identifiant les zones d'influence
        self.rstride, self.cstride = 5, 5

        s = ax.plot_surface(self.xx, self.yy, self.pdf, rstride=self.rstride, cstride=self.cstride, linewidth=.5, antialiased=True, color='gray', edgecolors='k')       
        a1 = s.__dict__['_original_facecolor']
        b1 = s.__dict__['_facecolors']
        c1 = s.__dict__['_facecolors3d']
        
        s = ax.plot_surface(self.xx, self.yy, self.pdf, rstride=self.rstride, cstride=self.cstride, linewidth=.5, antialiased=True, color='w', edgecolors='k')
        a2 = s.__dict__['_original_facecolor']
        b2 = s.__dict__['_facecolors']
        c2 = s.__dict__['_facecolors3d']
        
        Lx = int(self.nx/self.rstride)
        Ly = int(self.ny/self.cstride)

        mask = resize(self.masque, (Lx,Ly), order=0)
        indx = np.argwhere(mask)
        idx = indx[:,0]*Lx + indx[:,1]

        a = a1
        b = b1
        c = c1
        for i in idx:
            a[i,:] = a2[i,:]
            b[i,:] = b2[i,:]
            c[i,:] = c2[i,:]
        s.__dict__['_original_facecolor'] = a
        s.__dict__['_facecolors'] = b
        s.__dict__['_facecolors3d'] = c


    
    

    # Affiche la fonction de distribution normale des probabilités en 3D et ses contours en 2D
    def affiche_PDF_avec_contours(self, contours_remplis=True, affiche_labels=True, affiche_tickmarks=True, 
                                  view=[20., -20.], offset = -0.15, nom_figure=None):

        # Permet l'affichage de l'azimuth et de l'élévation lors de l'ajustement de l'angle de vue avec la souris.
        fig = plt.figure(figsize = (15,10))
        fig.canvas.set_window_title('3D')
        ax = fig.gca(projection='3d')

        # Génération de la surface 3D en 2 couleurs identifiant les zones d'influence
        self.genere_surf_3D(ax)
    
        # Contours 2D remplis en dessous
        if (contours_remplis==True):
            cset = ax.contourf(self.xx, self.yy, self.pdf, zdir='z', offset=offset, cmap='viridis')
        else:
            cset = ax.contour(self.xx, self.yy, self.pdf, zdir='z', offset=offset, levels = 10, cmap='hot') 
            
        # Affiche frontières entre les zones d'influence
        ax.contour(self.xx, self.yy, self.masque, [0.5], offset=offset, linewidths=2., colors='white') 

        ax.text(2, 1, offset,'$\mathcal{R}_{1}$', zdir='y', color='w',fontsize=18)        
        ax.text(9, 8, offset,'$\mathcal{R}_{2}$', zdir='y', color='w',fontsize=18)    
        ax.text(6, 2.5, 0.05, '$p(\mathbf{X}|w_{1})p(w_{1})$', zdir='y', horizontalalignment='right', verticalalignment='bottom', color='k',fontsize=18)    
        ax.text(6, 6.5, 0.08, '$p(\mathbf{X}|w_{2})p(w_{2})$', zdir='y', horizontalalignment='right', verticalalignment='bottom', color='gray',fontsize=18)    
        
        ax.set_zlim(offset,np.max(self.pdf))

        ax.view_init(view[0], view[1])

        if (affiche_labels==True):
            ax.set_ylabel('$x_{2}$', fontsize=18)
            ax.xaxis.set_rotate_label(False)  
            ax.set_xlabel('$x_{1}$', rotation=10, fontsize=18)
            

        if (affiche_tickmarks==False):
            # Enlève tickmarks
            ax.xaxis.set_major_formatter(NullFormatter())
            ax.yaxis.set_major_formatter(NullFormatter())
            ax.zaxis.set_major_formatter(NullFormatter())
            
        fig.tight_layout()
        
        
        # Sauvegarde de l'image 
        if nom_figure!=None:
            plt.savefig(nom_figure, format="svg")
            
            
        # Affiche azimuth et élévation lorsque l'on change l'angle de vue avec la souris
        def on_click(event):
            print('view= [%.1f , %.1f]' % (ax.elev, ax.azim))   
            
        cid = fig.canvas.mpl_connect('button_release_event', on_click)
        
        
        plt.show()
    



In [3]:
if __name__=='__main__':

    # ------- Paramètres des gaussiennes --------
    mu = np.zeros((2,2))
    mu[0,:] = [6., 3.]
    mu[1,:] = [6., 7.]

    sigma = np.zeros((2,2))
    sigma[0,:] = [2., 0.7]
    sigma[1,:] = [1.2, 0.7]

    angle = np.array([-30., 0.]) 
    prob_C = np.array([0.5, 0.5]) 

    
    # ----------- Génération des distributions --------------
    pdf = genere_distributions(mu, sigma, angle, prob_C)
    
    
    # -----------Affichage des distributions ---------
    # Utiliser la commande suivante pour déterminer les valeurs optimales d'angles de vue (avec la souris) et d'offset    
    #pdf.affiche_PDF_avec_contours(offset = -0.05)
    
    # Affichage et sauvegarde des meilleurs résultats
    nom_figure = "Figure_15_41.svg"
    pdf.affiche_PDF_avec_contours(offset = -0.05, view=[15., -19], affiche_tickmarks=False, nom_figure=nom_figure)

    